In [3]:
import re

yellow_color = "\033[93m"
reset_color = "\033[00m"

class CallStack:
    def __init__(self):
        self.stack = []
        self.call_graph = {}
        self.function_locations = {}
        self.root_functions = set()

    def process_log_line(self, line):
        # 提取函数名和路径：行号        
        call_match = re.search(r'call function (\w+) in (.+:\d+)', line)
        if call_match:
            function_name = call_match.group(1)
            location = call_match.group(2)
            self.stack.append(function_name)
            if len(self.stack) > 1:
                caller = self.stack[-2]
                if caller not in self.call_graph:
                    self.call_graph[caller] = []
                if function_name not in self.call_graph[caller]:
                    self.call_graph[caller].append(function_name)
            else:
                self.root_functions.add(function_name)
            self.function_locations[function_name] = location
        elif 'exit function' in line:
            if self.stack:
                self.stack.pop()

    def process_log(self, log_lines):
        for line in log_lines:
            self.process_log_line(line)

    def print_call_tree(self):
        visited = set()

        def print_subtree(function_name, indent="", is_last=True):
            if function_name in visited:
                return
            visited.add(function_name)
            location = self.function_locations.get(function_name, "")
            connector = "└── " if is_last else "├── "
            # print(f"{indent}{connector}{yellow_color}{function_name}{reset_color}, {location}")
            print(f"{indent}{connector}{function_name}, {location}")
            indent += "    " if is_last else "│   "
            if function_name in self.call_graph:
                child_functions = self.call_graph[function_name]
                for i, callee in enumerate(child_functions):
                    is_last_callee = (i == len(child_functions) - 1)
                    print_subtree(callee, indent, is_last_callee)

        root_functions = list(self.root_functions)
        for i, root_function in enumerate(root_functions):
            is_last_root = (i == len(root_functions) - 1)
            print_subtree(root_function, is_last=is_last_root)

# 读取日志文件并处理
with open('./logs/logs-finetune_4D-host_7c6a3d21f7e2-pid_52125-py/tracing-finetune_4D-20240821_144339.log', 'r') as log_file:
    log_data = log_file.readlines()

# 处理日志并打印调用树
call_stack = CallStack()
call_stack.process_log(log_data)
call_stack.print_call_tree()

├── get_all_model_patch, /root/vescale_prj/veScale/vescale/model/patch/__init__.py:23
├── __init__, /root/vescale_prj/veScale/vescale/dtensor/placement_types.py:2
│   ├── update_vescale_debug_mode_from_env, /root/vescale_prj/veScale/vescale/debug/debug_log.py:88
│   ├── _get_or_create_default_group, /root/vescale_prj/veScale/vescale/dtensor/device_mesh.py:313
│   │   └── _get_device_handle, /root/vescale_prj/veScale/vescale/dtensor/device_mesh.py:153
│   ├── _get_current_device, /root/vescale_prj/veScale/vescale/dtensor/device_mesh.py:281
│   ├── _validate_mesh, /root/vescale_prj/veScale/vescale/dtensor/device_mesh.py:339
│   ├── _init_process_groups, /root/vescale_prj/veScale/vescale/dtensor/device_mesh.py:364
│   │   └── get_rank, /root/vescale_prj/veScale/vescale/dtensor/device_mesh.py:498
│   ├── rng_states, /root/vescale_prj/veScale/vescale/dtensor/random.py:136
│   ├── __post_init__, /root/vescale_prj/veScale/vescale/dtensor/op_schema.py:133
│   ├── get_dim_groups, /root/vescale_

In [1]:
import re

class Node:
    def __init__(self, name):
        self.name = name
        self.children = []
        self.visited = False

    def add_child(self, child):
        # 检查是否存在重复子节点
        for existing_child in self.children:
            if existing_child.name == child.name:
                return
        self.children.append(child)

def parse_log_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    root = Node("root")
    stack = [root]

    for line in lines:
        if not line.strip():
            continue

        indent_level = (line.count('-') // 4)

        # 提取函数名和路径：行号
        call_match = re.search(r'call function (\w+) in (.+:\d+)', line)
        # exit_match = re.search(r'exit function (\w+) in (.+:\d+)', line)

        if call_match:
            function_name = call_match.group(1)
            path_line = call_match.group(2)
            line_content = f"{function_name}, {path_line}"
        # elif exit_match:
        #     function_name = exit_match.group(1)
        #     path_line = exit_match.group(2)
        #     line_content = f"exit {function_name} in {path_line}"
        else:
            continue

        node = Node(line_content)

        while len(stack) > indent_level + 1:
            stack.pop()

        # 检查重复调用
        if not any(child.name == node.name for child in stack[-1].children):
            stack[-1].add_child(node)
            stack.append(node)

    return root

def print_tree(node, prefix=""):
    children = node.children
    for index, child in enumerate(children):
        connector = "└── " if index == len(children) - 1 else "├── "
        print(prefix + connector + child.name)
        if child.children:
            extension = "    " if index == len(children) - 1 else "│   "
            print_tree(child, prefix + extension)

# 读取日志文件并生成树结构
file_path = '/root/vescale_prj/veScale/examples/nanogpt_4D_finetune/logs-finetune_4D-host_7c6a3d21f7e2-pid_52125-py/tracing-finetune_4D-20240821_144339.log'
# file_path = 'path_to_log_file.log'  # 替换为日志文件的实际路径
root = parse_log_file(file_path)

# 打印树状结构
print_tree(root)

└── main, /root/vescale_prj/veScale/examples/nanogpt_4D_finetune/finetune_4D.py:195
    ├── init_device_mesh, /root/vescale_prj/veScale/vescale/devicemesh_api/api.py:48
    ├── init_device_mesh, /root/vescale_prj/veScale/vescale/dtensor/device_mesh.py:594
    │   ├── __init__, /root/vescale_prj/veScale/vescale/dtensor/device_mesh.py:224
    │   ├── update_vescale_debug_mode_from_env, /root/vescale_prj/veScale/vescale/debug/debug_log.py:88
    │   ├── _get_or_create_default_group, /root/vescale_prj/veScale/vescale/dtensor/device_mesh.py:313
    │   │   └── _get_device_handle, /root/vescale_prj/veScale/vescale/dtensor/device_mesh.py:153
    │   ├── _get_device_handle, /root/vescale_prj/veScale/vescale/dtensor/device_mesh.py:153
    │   ├── _get_current_device, /root/vescale_prj/veScale/vescale/dtensor/device_mesh.py:281
    │   ├── _validate_mesh, /root/vescale_prj/veScale/vescale/dtensor/device_mesh.py:339
    │   └── _init_process_groups, /root/vescale_prj/veScale/vescale/dtensor/devic